In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings (optional, but improves readability)
pd.set_option('display.max_colwidth', None)


In [67]:
df = pd.read_csv("../data/transactions.csv")
df_original = pd.read_csv("../data/transactions.csv")

print(f"Number of transactions : {df.shape[0]}")
df.head(5)

Number of transactions : 451


,description,amount,label,balance,date
0,Fusion Interest Rebate,22.85,Interest Received,15737.35,2023-10-10 00:00:00
1,Rtc Express Pmt To Battlezone Party Lance 3 Sep,-650.0,Entertainment,44479.06,2023-08-31 00:00:00
2,POS PURCHASE (EFFEC ) DJS BILTONG (PTY) LTD NORKE CARD NO.,-90.00,Fast Food & Takeouts,38 624.93,2023-10-27T00:00:00
3,POS Purchase Engen Mitchell Park * 07 Oct,-377.17,Fuel,59485.18,2023-10-10 00:00:00
4,POS Purchase Woolworths Online * 23 Aug,-1531.85,Groceries,72873.35,2023-08-25 00:00:00


### Investigating the `balance` Column

At this stage, I wanted to understand the role and reliability of the `balance` column in the dataset.

Since each transaction includes an `amount` and a `balance`, I hypothesized that the `balance` might represent a **running total** — the account balance after each transaction.

To validate this, I:
- Checked the data type of `balance`, and found it was stored as an `object` (i.e., string), which is common in CSV files containing formatted numbers.
- Cleaned the `balance` values by removing spaces and commas.
- Converted the column to `float` so mathematical operations could be performed.
- Calculated the difference between consecutive `balance` entries (`balance_diff`) to simulate how the balance changes over time.
- Compared this `balance_diff` to the `amount` column to see if they matched.

This investigation helped assess whether `balance` could be used to support or verify the transaction flow logic.


In [57]:
df.sort_values(by="date", ascending=True, inplace=True)
df.head()

,description,amount,label,balance,date,balance_diff
149,SMS Notification Fee,-0.80,Bank Charges and Fees,26.72,2021-07-25T00:00:00,NaN
18,Immediate Payment Fee,-7.50,Bank Charges and Fees,6005.20,2021-07-28T00:00:00,5978.48
243,Markham Springs (Card ),-600.00,Clothing,3235.70,2021-07-28T00:00:00,-2769.50
215,SMS Notification Fee,-3.60,Bank Charges and Fees,5293.10,2021-07-28T00:00:00,2057.40
97,Pizza Hut Springs Mall Springs (Card ),-55.80,Fast Food & Takeouts,3835.70,2021-07-28T00:00:00,-1457.40


In [58]:
print(df['balance'].dtype)

float64


In [59]:
#df['balance'] = df['balance'].str.replace(' ', '').str.replace(',', '') ALready Applied
df['balance'] = df['balance'].astype(float)
print(df['balance'].dtype)
print(df['balance'].head())

float64
149      26.72
18     6005.20
243    3235.70
215    5293.10
97     3835.70
Name: balance, dtype: float64


In [60]:
df['balance_diff'] = df['balance'].diff()

In [61]:
df[['amount', 'balance', 'balance_diff']].head(10)

,amount,balance,balance_diff
149,-0.80,26.72,NaN
18,-7.50,6005.20,5978.48
243,-600.00,3235.70,-2769.50
215,-3.60,5293.10,2057.40
97,-55.80,3835.70,-1457.40
189,-450.00,5555.20,1719.50
188,-0.40,4699.00,-856.20
279,-1.20,1630.43,-3068.57
361,-3.50,931.68,-698.75
63,-2 300.52,935.18,3.50


### Conclusion on the `balance` Column

The analysis showed that the differences in `balance` values (`balance_diff`) often did **not** match the `amount` values from the same row. In some cases, the balance increased even when the transaction amount was negative.

This suggests that:
- The `balance` column does **not** represent a reliable running total of a single account.
- The data might include transactions from **multiple accounts**, or the `balance` values could be **anonymized or simulated**.
- As a result, the `balance` column is not useful for guiding or validating token-replacement logic in this project.

For now, I will **exclude `balance` from preprocessing** and focus on the `description` and `label` columns, which are the core elements of the task.


In [69]:
# df.drop('balance_diff', axis=1, inplace=True) Already Run this line
df = df_original
df.head(10)


,description,amount,label,balance,date
0,Fusion Interest Rebate,22.85,Interest Received,15737.35,2023-10-10 00:00:00
1,Rtc Express Pmt To Battlezone Party Lance 3 Sep,-650.0,Entertainment,44479.06,2023-08-31 00:00:00
2,POS PURCHASE (EFFEC ) DJS BILTONG (PTY) LTD NORKE CARD NO.,-90.00,Fast Food & Takeouts,38 624.93,2023-10-27T00:00:00
3,POS Purchase Engen Mitchell Park * 07 Oct,-377.17,Fuel,59485.18,2023-10-10 00:00:00
4,POS Purchase Woolworths Online * 23 Aug,-1531.85,Groceries,72873.35,2023-08-25 00:00:00
5,POS Purchase Engen Nkandla * 05 Oct,-27.85,General Purchases,74154.54,2023-10-07 00:00:00
6,SMS Notification Fee Branch:,470010 -0.80,Bank Charges and Fees,84.08,2021-10-03T00:00:00
7,Banking App Payment,-200.00,Bank Transfer,111.86,2021-09-28T00:00:00
8,Eft Debit Order Payment (): Cartrack,-199.00,Transport,2 887.60,2021-10-25T00:00:00
9,Magtape Debit MTN Sp,-1797.99,Cellular Data Purchase,6809.59,2023-08-31 00:00:00


### Exploring the `label` Column

To better understand the target classes for classification, I explored the `label` column. This revealed that:



In [ ]:
unique_count = df['label'].nunique()
value_counts = df['label'].value_counts()

print(f"Number of Labels: {unique_count}")
print(value_counts)


Number of Labels: 44
label
Bank Charges and Fees         68
Eating Out                    52
Insufficient Funds Fee        45
Groceries                     44
General Purchases             35
Subscriptions                 20
Cellular Data Purchase        16
Fuel                          11
Coffee                        11
Transport                      9
Bank Transfer                  9
Entertainment                  8
Savings Transfer               8
Home and Garden                8
ATM and Cash Withdrawal        7
Debit Order                    7
Gambling                       7
Parking                        7
Fast Food & Takeouts           6
Investments                    6
Clothing                       6
Life Insurance                 5
Health and Medical             5
Tobacco & Smoking              5
Interest Received              4
Alcohol                        4
Personal Care                  4
Interest Repayment             4
General Insurance              4
General Loan Rep

- There are **44 unique labels** in total.
- Some labels are quite frequent, such as:
  - `"Bank Charges and Fees"` (68 instances)
  - `"Eating Out"` (52 instances)
  - `"Groceries"` (44 instances)
- Others are extremely rare, with only one or two examples:
  - `"Sport"`, `"Donation"`, `"Cash Deposit Fee"`

This label imbalance may affect classification performance later. For now, it helps me prioritize which categories to target in the token-replacement logic — starting with the high-frequency ones where patterns are most consistent and impactful.


###  Exploring `description` Patterns – Label: Groceries

I explored 10 sample descriptions where the label was `Groceries`.




In [75]:
df[df['label'] == 'Groceries']['description'].sample(10, random_state=1)


26     POS PURCHASE (EFFEC ) PnP Fam Elgin KEMPT CARD NO. 
24              POS Purchase PNP Fam Craighall P  * 24 Aug
299             POS Purchase PNP Fam Craighall P  * 14 Nov
249             POS Purchase Spar Craighall Park  * 27 Oct
258    POS PURCHASE (EFFEC ) PnP Fam Elgin KEMPT CARD NO. 
408             POS Purchase Umhlali Fresh Produ  * 18 Aug
221             POS Purchase Checkers Fresh X Ol  * 21 Sep
351                      POS Purchase Woolworths  * 15 Oct
336    POS PURCHASE (EFFEC ) PnP Fam Elgin KEMPT CARD NO. 
333               POS Purchase Superspar Harding  * 18 Oct
Name: description, dtype: object

Key patterns observed:
- All entries began with `"POS Purchase"`, suggesting they were point-of-sale transactions.
- Common merchant names included `"PNP"`, `"Checkers"`, `"Spar"`, `"Woolworths"`, and `"Superspar"`, indicating these can be categorized under a `[Grocer]` class token.
- Many descriptions ended with a date-like format (e.g. `"* 14 Nov"`), which can be replaced with `[Date]`.
- Intermediate tokens may represent branch or location names, which could later be abstracted into `[Location]` if reliable rules are defined.

These observations will guide the construction of rules for replacing specific tokens in transaction descriptions with semantically meaningful class tokens.

In [ ]:
df[df['label'] == 'Eating Out']['description'].sample(10, random_state=1)

266    POS Purchase Fournos Bakery Dunk  * 17 Sep
37      POS Purchase Steers Ultra South  * 25 Aug
356            POS Purchase Nice On 4th  * 08 Sep
267              POS Purchase Uber Eats  * 21 Oct
264        POS Purchase KFC Grayston 15  * 12 Oct
377    POS Purchase Cato Ridge One Stop  * 17 Sep
272    POS Purchase Marble Pantry - Arc  * 07 Sep
319    POS Purchase Zapper *Mamma Gs Uc  * 09 Sep
375     POS Purchase KFC Castle Gate 46  * 01 Sep
165    POS Purchase Marble Pantry - Arc  * 22 Aug
Name: description, dtype: object

In [78]:
df[df['label'] == 'Fuel']['description'].sample(10, random_state=1)

156    Fuel Purchase Engen Big Bird Sout  * 16 Sep
174    Fuel Purchase Engen Big Bird 1 St  * 25 Aug
210          Fuel Purchase Sasol Melmoth  * 02 Oct
434      Fuel Purchase Sasol Maathula  S  * 03 Oct
109    Fuel Purchase Shell Parkhurst Cor  * 16 Nov
329    Fuel Purchase Shell Parkhurst Cor  * 19 Nov
3       POS Purchase Engen Mitchell Park  * 07 Oct
348     POS Purchase Shell Parkhurst Cor  * 30 Oct
445    Fuel Purchase Cj Motors(Engen Ser  * 19 Sep
431    Fuel Purchase Shell Parkhurst Cor  * 27 Aug
Name: description, dtype: object

After examining sample descriptions for `Eating Out` and `Fuel`, several structural patterns emerged:

#### 🍔 Eating Out:
- Consistently begins with `"POS Purchase"`
- Contains clear food-related vendor names such as `"KFC"`, `"Uber Eats"`, `"Steers"`
- Includes location/branch names and often ends with a date
- Can be generalized to:  
  `POS Purchase [Restaurant] [Location] [Date]`

#### ⛽ Fuel:
- Begins with `"Fuel Purchase"` or `"POS Purchase"`
- Contains fuel station names like `"Engen"`, `"Shell"`, `"Sasol"`
- Includes branch/location info and ends with a date
- Can be generalized to:  
  `Fuel Purchase [FuelBrand] [Location] [Date]`

These insights help define specific replacement rules for class tokens like `[Restaurant]`, `[FuelBrand]`, `[Location]`, and `[Date]`.